# <center> Dockless Dataset </center>
### <center> Extract, Transform, Load </center>

In [1]:
import pandas as pd
from pandas.io.json import json_normalize
from pandas import Timestamp
from datetime import datetime, timedelta
import pymongo

### Mongo Initialization
##### Database ⟷ dockless_vehicle_db & Collection ⟷ dockless_collection 

In [2]:
conn = 'mongodb://127.0.0.1:27017' # Initialize PyMongo to work with MongoDBs
client = pymongo.MongoClient(conn) # Set Client connection to server (daemon)

db = client.dockless_vehicle_db          # Select MongoDB database to work with
collection = db.dockless_collection      # Collection Title

### EXTRACTION :

In [3]:
csv_filepath = '../dev/Resources/Dockless_Vehicle_Trips.csv' #Omar Filepath
#csv_filepath = '../../project-ETL/Dockless_Vehicle_Trips.csv' #Michael Filepath

dockless_data_df = pd.read_csv(csv_filepath)
dockless_data_df = dockless_data_df.rename(columns={'Trip Duration':'Duration (Seconds)', 
                                                    'Trip Distance':'Distance (Meters)'})
dockless_data_df.head()

/Users/Omar/anaconda3/envs/pythonomy/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,Device ID,Vehicle Type,Duration (Seconds),Distance (Meters),Start Time,End Time,Modified Date,Month,Hour,Day of Week,Council District (Start),Council District (End),Origin Cell ID,Destination Cell ID,Year,Start Latitude,Start Longitude,End Latitude,End Longitude
0,000000cb-78a9-4b95-8ead-37c797abcf37,ad14e63b-940d-4ca3-84c6-0131147fba45,scooter,358.0,915.0,06/29/2018 05:30:00 PM,06/29/2018 05:30:00 PM,12/12/2018 09:36:14 PM,6.0,17.0,5.0,5.0,9.0,013445,013916,2018.0,30.263,-97.762,30.259,-97.754
1,00001b5f-46aa-4b26-a12f-d7b4d3c69834,c5a230c9-7e80-4464-862f-f2b23f61d6d3,scooter,226.0,839.0,11/30/2018 11:00:00 AM,11/30/2018 11:00:00 AM,12/13/2018 03:43:44 PM,11.0,11.0,5.0,9.0,9.0,014556,015186,2018.0,30.283,-97.745,30.284,-97.737
2,00002334-2f2b-41a0-9a04-8ff66be39f40,5a9f7815-bf0c-4fef-8bac-60efd4f23b10,scooter,324.0,1206.0,10/02/2018 03:15:00 PM,10/02/2018 03:30:00 PM,12/13/2018 03:49:54 PM,10.0,15.0,2.0,9.0,9.0,015344,015032,2018.0,30.285,-97.734,30.291,-97.738
3,0000261e-c0f0-4fb1-9947-1606dfe4ddcf,41fa0cf0-f433-4b3a-89f5-2c6cec6c367d,scooter,1096.0,0.0,09/02/2018 04:45:00 PM,09/02/2018 05:00:00 PM,12/13/2018 03:53:09 PM,9.0,16.0,0.0,9.0,9.0,014707,014081,2018.0,30.268,-97.743,30.276,-97.752
4,00002c70-678b-4c09-a32c-da0266e329c6,ddd4bbc3-114e-4cbc-b078-860e3c28b11b,scooter,408.0,1144.0,09/28/2018 11:45:00 AM,09/28/2018 11:45:00 AM,12/12/2018 10:24:56 PM,9.0,11.0,5.0,3.0,3.0,016114,015642,2018.0,30.242,-97.721,30.244,-97.729


### TRANSFORMATION :

In [4]:
dockless_data_df = dockless_data_df[["ID", "Vehicle Type", "Duration (Seconds)", "Distance (Meters)", "Start Time" ]].copy()
dockless_data_df.head() #ONLY NEEDED COLUMNS COPIED OVER

,ID,Vehicle Type,Duration (Seconds),Distance (Meters),Start Time
0,000000cb-78a9-4b95-8ead-37c797abcf37,scooter,358.0,915.0,06/29/2018 05:30:00 PM
1,00001b5f-46aa-4b26-a12f-d7b4d3c69834,scooter,226.0,839.0,11/30/2018 11:00:00 AM
2,00002334-2f2b-41a0-9a04-8ff66be39f40,scooter,324.0,1206.0,10/02/2018 03:15:00 PM
3,0000261e-c0f0-4fb1-9947-1606dfe4ddcf,scooter,1096.0,0.0,09/02/2018 04:45:00 PM
4,00002c70-678b-4c09-a32c-da0266e329c6,scooter,408.0,1144.0,09/28/2018 11:45:00 AM


In [5]:
dockless_data_df["Start Time"][:2].values #testing element
#dockless_data_df["Start Time"][0].strip(' PM') #testing element

array(['06/29/2018 05:30:00 PM', '11/30/2018 11:00:00 AM'], dtype=object)

In [6]:
dockless_data_df = dockless_data_df.dropna(subset=["Start Time"])

In [7]:
def round_nearest_hour(input_time):
    try:
        time = datetime.strptime(input_time, "%m/%d/%Y %I:%M:%S %p")
   #    time = datetime.strptime(input_time, "%m/%d/%Y %H:%M")
        time = time + timedelta(minutes=30)
        month = time.month
        year = time.year
        day = time.day
        hour = time.hour
        minute = 0
        return datetime(year, month, day, hour, minute)
    except:
        print("Error: ", input_time)

In [8]:
round_nearest_hour('06/29/2018 05:30:00 PM') #FUNCTION TEST
#round_nearest_hour('6/29/2018 23:20') #FUNCTION TEST

datetime.datetime(2018, 6, 29, 18, 0)

In [9]:
dockless_data_df["Start Time (rounded)"] = dockless_data_df["Start Time"].apply(round_nearest_hour)
dockless_data_df.head()

,ID,Vehicle Type,Duration (Seconds),Distance (Meters),Start Time,Start Time (rounded)
0,000000cb-78a9-4b95-8ead-37c797abcf37,scooter,358.0,915.0,06/29/2018 05:30:00 PM,2018-06-29 18:00:00
1,00001b5f-46aa-4b26-a12f-d7b4d3c69834,scooter,226.0,839.0,11/30/2018 11:00:00 AM,2018-11-30 11:00:00
2,00002334-2f2b-41a0-9a04-8ff66be39f40,scooter,324.0,1206.0,10/02/2018 03:15:00 PM,2018-10-02 15:00:00
3,0000261e-c0f0-4fb1-9947-1606dfe4ddcf,scooter,1096.0,0.0,09/02/2018 04:45:00 PM,2018-09-02 17:00:00
4,00002c70-678b-4c09-a32c-da0266e329c6,scooter,408.0,1144.0,09/28/2018 11:45:00 AM,2018-09-28 12:00:00


In [10]:
dockless_data_df = dockless_data_df[["ID", "Vehicle Type", "Duration (Seconds)", "Distance (Meters)", "Start Time (rounded)" ]].copy()
dockless_data_df.head()

,ID,Vehicle Type,Duration (Seconds),Distance (Meters),Start Time (rounded)
0,000000cb-78a9-4b95-8ead-37c797abcf37,scooter,358.0,915.0,2018-06-29 18:00:00
1,00001b5f-46aa-4b26-a12f-d7b4d3c69834,scooter,226.0,839.0,2018-11-30 11:00:00
2,00002334-2f2b-41a0-9a04-8ff66be39f40,scooter,324.0,1206.0,2018-10-02 15:00:00
3,0000261e-c0f0-4fb1-9947-1606dfe4ddcf,scooter,1096.0,0.0,2018-09-02 17:00:00
4,00002c70-678b-4c09-a32c-da0266e329c6,scooter,408.0,1144.0,2018-09-28 12:00:00


In [11]:
# TOP 100 ROWS FOR TESTING; ORIGINAL DATASET CONTAINS 2.6 MILLION ROWS;
dockless_data_top100_df = dockless_data_df[:][:100]
len(dockless_data_top100_df)

100

### LOADING: 
#### INTO MONGODB

In [13]:
# LIMITED DATASET OF TOP 100 ROWS FOR FUNCTIONALITY TESTING
for index, row in dockless_data_top100_df.iterrows():
    data = row.to_dict()
    print(data)
    print('\n––––––––––––––––––––––––––––––\n')
    collection.insert_one(data)

{'ID': '000000cb-78a9-4b95-8ead-37c797abcf37', 'Vehicle Type': 'scooter', 'Duration (Seconds)': 358.0, 'Distance (Meters)': 915.0, 'Start Time (rounded)': Timestamp('2018-06-29 18:00:00')}

––––––––––––––––––––––––––––––

{'ID': '00001b5f-46aa-4b26-a12f-d7b4d3c69834', 'Vehicle Type': 'scooter', 'Duration (Seconds)': 226.0, 'Distance (Meters)': 839.0, 'Start Time (rounded)': Timestamp('2018-11-30 11:00:00')}

––––––––––––––––––––––––––––––

{'ID': '00002334-2f2b-41a0-9a04-8ff66be39f40', 'Vehicle Type': 'scooter', 'Duration (Seconds)': 324.0, 'Distance (Meters)': 1206.0, 'Start Time (rounded)': Timestamp('2018-10-02 15:00:00')}

––––––––––––––––––––––––––––––

{'ID': '0000261e-c0f0-4fb1-9947-1606dfe4ddcf', 'Vehicle Type': 'scooter', 'Duration (Seconds)': 1096.0, 'Distance (Meters)': 0.0, 'Start Time (rounded)': Timestamp('2018-09-02 17:00:00')}

––––––––––––––––––––––––––––––

{'ID': '00002c70-678b-4c09-a32c-da0266e329c6', 'Vehicle Type': 'scooter', 'Duration (Seconds)': 408.0, 'Distance

In [ ]:
# DO NOT RUN
# COMPLETE DATASET OF 2.6 MILLION ROWS
for index, row in dockless_data_df.iterrows():
    data = row.to_dict()
    collection.insert_one(data)